In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Imports

In [3]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import time
from tqdm import tqdm

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.max_rows', 500)

In [5]:
from waad.utils.tuples_analyser import AnalystTuplesAnalyser
from waad.utils.constants import TupleAnalysisFields
from waad.utils.data import Data
from waad.utils.combinations_utils import flatten
from waad.utils.postgreSQL_utils import Database, Table
from waad.utils.tuples_big_data import Cache, ComputeMutualInfoOnGivenLevel, ComputeMutualInfoScoreGroupings, ComputeScoreOnGivenLevel, ScoreGroupings

In [6]:
from ad_tree.array_record import ArrayRecord
from ad_tree.file_record import FileRecord
from ad_tree.sparse_ADTree import ADNode
from ad_tree.iterated_tree_contingency_table import ContingencyTable

## Prepare database

In [7]:
HOST = '127.0.0.1' 
PORT = '5432'
USER = ''   # To fill
PASSWORD = ''   # To fill
DB_NAME = ''   # To fill
TABLE_NAME = ''  # To fill

In [8]:
db = Database(HOST, PORT, USER, PASSWORD, DB_NAME)
table = Table(db, TABLE_NAME)

In [9]:
# Comment and uncomment the meta_fields you want to use

META_FIELDS = [('subjectusersid', 'subjectusername', 'subjectdomainname'),
 ('targetusersid', 'targetusername', 'targetdomainname'),
 ('targetservername', 'targetinfo'),
 ('failurereason', 'status', 'substatus'),
 'logontype',
 'eventid',
 'ipaddress',
 'logonprocessname',
 ('authenticationpackagename', 'lmpackagename'),
 'workstationname',
 'host']

## Example of request

In [10]:
number_lines = 1000000
request = f"SELECT {', '.join(['eventrecordid', 'systemtime'] + flatten(META_FIELDS))} FROM {table.table_name} WHERE eventid = 4624 FETCH FIRST {number_lines} ROWS ONLY;"
data = table.get_command(request)

## Filtering bloc

In [11]:
# data = Data.filter_dataframe_field_on_value(data, 'logontype', 3)

## Pre-processing

In [12]:
N = data.shape[0]

**Build meta fields**

In [13]:
Data.build_meta_fields(data, META_FIELDS)

**Convert to categorical**

If we have data to 'categorise', an additional dictionnary parameter is needed : 

```
{
 'trade_usd': [0, 100000, 1000000, 10000000, 100000000, 1000000000, 10000000000, 100000000000, 1000000000000, 10000000000000], 
 'weight_kg': [0, 100000, 1000000, 10000000, 100000000, 1000000000, 10000000000, 100000000000, 1000000000000, 10000000000000]
}
```
It contains the different bins that we want to define on the columns to categorize.

In [14]:
converter = Data.set_as_categorical(data, META_FIELDS, additional_info={})

**Get records table in categorial format**

In [15]:
records_table = np.transpose([data[field].cat.codes for field in META_FIELDS]).tolist()

**Get arity list**

In [16]:
arity_list = [len(dict(enumerate(data[f].cat.categories))) for f in META_FIELDS]

## Building ADTree

In [ ]:
start = time.time()

array_record = ArrayRecord(arity_list, records_table)

# build an AD-Tree with attribute list starts from the first attribute and for all the records
adtree = ADNode(1, record_nums=list(range(1, array_record.records_length + 1)), array_record=array_record)

print(time.time() - start)

## Build cache

In [18]:
MAX_LEVEL = 4

In [ ]:
cache = Cache(adtree, META_FIELDS, MAX_LEVEL)
cache.run()

# Manual Pipeline

A first solution is to execute the pipeline step by step manually, to tune the parameter `mu` on each level.

## Compute mutual information

In [ ]:
CMIOGL = {}

for level in range(2, MAX_LEVEL + 1):
    cmiogl = ComputeMutualInfoOnGivenLevel(cache, level)
    cmiogl.run()
    cmiogl.plot_mutual_info()
    CMIOGL[level] = cmiogl

In [21]:
MU = {}

MU[2] = 0.1
MU[3] = 0.1
MU[4] = 0.1
# MU[5] = 0.1

In [22]:
PAIRINGS_TO_KEEP = {}

for level in range(2, MAX_LEVEL + 1):
    PAIRINGS_TO_KEEP[level] = CMIOGL[level].get_pairings_to_keep(MU[level])

## Scores computation 

In [23]:
T_ALPHA = 20

In [24]:
CSOGL = {}

for level in range(2, MAX_LEVEL + 1):
    csogl = ComputeScoreOnGivenLevel(cache, level, T_ALPHA, PAIRINGS_TO_KEEP[level])
    csogl.run()
    CSOGL[level] = csogl 

## ScoreGrouping

In [ ]:
FIRSTS_N = 10

sg = ScoreGroupings(CSOGL, max_level=4, firsts_n=FIRSTS_N, converter=converter)
sg.run()

for index, score_group in enumerate(sg.score_groupings):
    print(f'ScoreGroup index : {index}')
    score_group.display()

In [26]:
ComputeMutualInfoScoreGroupings.save_static(
        path='./save.json', 
        table_name=table.table_name, 
        psql_request=request, 
        meta_fields=META_FIELDS, 
        t_alpha=T_ALPHA,
        mus=list(MU.values()),
        firsts_n=FIRSTS_N,
        csogls=CSOGL,
        score_groups=sg,
        converter=converter
    )

In [ ]:
ComputeMutualInfoScoreGroupings.display_from_json('./save.json')

### Additional qualification

#### Most frequent subpairings

csogl.display_most_frequents_subpairings() can take as an input either the index of the score (corresponds to the ranking of the score among other tuples of same size) we want to study or directly the correponding object `Score`.

Ex:
* CSOGL[2].display_most_frequents_subpairings(score=sg.score_groupings[0].score_group[0], firsts_n=10, converter=converter)
* CSOGL[2].display_most_frequents_subpairings(index_number=0, firsts_n=10, converter=converter) car au sein de son niveau, le tuple est au rang 0

/!\ Be careful to input the desired level in CSOGL => see following block

In [28]:
score = sg.score_groupings[0].score_group[0]
level = len(score.A_a) + len(score.B_b)

In [ ]:
CSOGL[level].display_most_frequents_subpairings(score=score, firsts_n=10, converter=converter)

#### AnalystTuplesAnalyser

In the same way csogl.get_corresponding_authentications() can take as an input either the corresponding index or directly the object `Score` :

* CSOGL[2].get_corresponding_authentications(data, index_number=0, converter=converter)
* CSOGL[2].get_corresponding_authentications(data, score=sg.score_groupings[0].score_group[0], converter=converter)

/!\ Attention à bien mettre le niveau considéré dans CSOGL => le calcul est automatisé avec level, 2 cases au-dessus

In [30]:
corresponding_authentications = CSOGL[level].get_corresponding_authentications(data, score=score, converter=converter)

In [ ]:
ata = AnalystTuplesAnalyser(corresponding_authentications, exploratory_fields=META_FIELDS)
ata.run()
if ata.candidate is not None:
    ata.candidate.display_centered_summary()
else:
    print("No candidate found with AnalystTuplesAnalyser for those parameters")

# Automatic Pipeline

The other solution is to run every step in an automatic way by inputing all parameters at the beginning of the pipeline.

In [ ]:
cmisg = ComputeMutualInfoScoreGroupings(max_level=4, cache=cache, mu=0.1, t_alpha=20, firsts_n=10, converter=converter)
cmisg.run()

for index, score_group in enumerate(cmisg.sg.score_groupings):
    print(f'ScoreGroup index : {index}')
    score_group.display()

In [33]:
cmisg.save(path='./save.json', table_name=table.table_name, meta_fields=META_FIELDS, psql_request=request)

In [ ]:
ComputeMutualInfoScoreGroupings.display_from_json('./save.json')

### Additional qualification

#### Most frequent subpairings

In [ ]:
cmisg.display_most_frequents_subpairings(score=cmisg.sg.score_groupings[0].score_group[0], firsts_n=10, converter=converter)

#### AnalystTuplesAnalyser

In [36]:
corresponding_authentications = cmisg.get_corresponding_authentications(data, score=cmisg.sg.score_groupings[0].score_group[0], converter=converter)

In [ ]:
ata = AnalystTuplesAnalyser(corresponding_authentications, exploratory_fields=META_FIELDS)
ata.run()
if ata.candidate is not None:
    ata.candidate.display_centered_summary()
else:
    print("No candidate found with AnalystTuplesAnalyser for those parameters")